In [35]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np

Donnez une prédiction pour si Arthur 53 ans, fumeur, sportif, 175
cm, 85 kg, avec un taux de cholestérol au dessus de la normal et
un taux de glucose normal, une tension artérielle systolique dans
la moyenne et une pression sanguine diastolique correspondant
à la moyenne du 3e quartile (50%-75%) du jeu de données, est un
sujet à risques cardio-vasculaire.

id;age;cholesterol;gluc;smoke;alco;active;cardio;female;male;imc;pressure

In [9]:
DF_PATH ="../data/cardio_light_std.csv"

df = pd.read_csv(DF_PATH, sep=";", index_col="id")
# X = df.drop(columns=["cardio"])
# y = df["cardio"]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# model = LogisticRegression(max_iter=1000)
# model.fit(X_train, y_train)

# y_pred = model.predict(X_test)


Pour ajouter Arthur à prédire, il faut calculer l'IMC ainsi que la pression.

## IMC

### Indice de Masse Corporelle

Calcul de l'IMC :

$$
IMC = \frac{\text{poids (kg)}}{\left( \frac{\text{taille (cm)}}{100} \right)^2}
$$

​
Catégorisation médicale simplifiée :

|IMC	|Catégorie|	Valeur|
|-|-|-|
|< 25	|Normal	|1|
|25 ≤ IMC < 30	|Surpoids |	2|
|≥ 30	|Obésité |	3|

Arthur fait 85 kg et mesure 175cm.

In [12]:
arthur_imc = 85 / (175/100)**2
print("imc d'arthur : ", arthur_imc)

if arthur_imc < 25 :
    arthur_imc = 1
elif 25 <= arthur_imc < 30:  # Correction de la syntaxe
    arthur_imc = 2
else :
    arthur_imc = 3

print("catégorie IMC d'arthur : ", arthur_imc)

imc d'arthur :  27.755102040816325
catégorie IMC d'arthur :  2


---
## pressure

### Tension artérielle

| Diastolique (`ap_lo`) | Systolique (`ap_hi`) | Interprétation        | Code |
| -------------------- | --------------------- | --------------------- | ---- |
| < 80                | < 120                | Normale               | 1    |
| 80–89             | 120–139               | Élevée (à surveiller) | 2    |
| ≥ 90                | ≥ 140                  | Hypertension          | 3    |


Si la systolique ou la diastolique dépasse un seuil, on prend la catégorie la plus élevée.

Arthur possède une tension artérielle systolique dans
la moyenne et une pression sanguine diastolique correspondant
à la moyenne du 3e quartile (50%-75%) du jeu de données

In [19]:
DF_PATH = "../data/cardio_optimized.csv"
df = pd.read_csv(DF_PATH, sep=";", index_col="id")

# Obtenir les valeurs du 75e percentile
arthur_ap_lo = df['ap_lo'].quantile(0.75)
arthur_ap_hi = df['ap_hi'].quantile(0.75)

# Classifier la pression
def classifier_pressure(row):
    if row['ap_hi'] < 120 and row['ap_lo'] < 80:
        return 1  # Normale
    elif row['ap_hi'] < 140 and row['ap_lo'] < 90:
        return 2  # À surveiller
    else:
        return 3  # Hypertension

# Appliquer la classification pour Arthur
arthur_pressure = classifier_pressure({'ap_hi': arthur_ap_hi, 'ap_lo': arthur_ap_lo})

print("ap_lo :", arthur_ap_lo, "ap_hi :", arthur_ap_hi)
print("pressure d'Arthur :", arthur_pressure)

ap_lo : 90.0 ap_hi : 140.0
pressure d'Arthur : 3


In [58]:
arthur_data = {
    'age': [53],
    'cholesterol': [1], # Au-dessus de la normale
    'gluc': [0],        # Normal
    'smoke': [1],       # Fumeur
    'alco': [0],        # Non alcool
    'active': [1],      # Sportif
    'female': [0],      
    'male': [1],        # Homme
    'imc': [arthur_imc],     
    'pressure': [arthur_pressure]     
}
arthur = pd.DataFrame(arthur_data)

print("Shape de arthur_array :", arthur.shape)

Shape de arthur_array : (1, 10)


In [59]:
DF_PATH ="../data/cardio_light_std.csv"

df = pd.read_csv(DF_PATH, sep=";", index_col="id")
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 68562 entries, 0 to 99999
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          68562 non-null  float64
 1   cholesterol  68562 non-null  int64  
 2   gluc         68562 non-null  int64  
 3   smoke        68562 non-null  int64  
 4   alco         68562 non-null  int64  
 5   active       68562 non-null  int64  
 6   cardio       68562 non-null  int64  
 7   female       68562 non-null  float64
 8   male         68562 non-null  float64
 9   imc          68562 non-null  int64  
 10  pressure     68562 non-null  int64  
dtypes: float64(3), int64(8)
memory usage: 6.3 MB


In [ ]:
X = df.drop(columns=["cardio"])
y = df["cardio"]

model = LogisticRegression(max_iter=1000, C= 200, penalty= 'l2', solver= 'liblinear')
model.fit(X, y)


y_pred = model.predict(arthur)
probability = model.predict_proba(arthur)[:, 1]

y_pred[0], probability

(np.int64(1), array([1.]))

Pour rappel, voici les métrics du model utilisé : 

| Metrics | Scoring|
| :-:|:-: |
|Accuracy | 0.72 |
| Précision | 0.72 |
|Recall | 0.72 |
|Recall classe 1 | 0.69 |
| F1 Score | 0.72 |
